In [1]:
# import cell
%matplotlib inline
import glob
import cv2
import numpy as np
import scipy.misc as smp
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import csv
import re
import os
import json

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda, ELU
from keras.layers.convolutional import Convolution2D

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
# read csv and make data map

def get_image_name(name, regex):
    m = re.search(regex, name)
    if m:
        return m.group(1)
    
def read_data_file(path):
    # http://www.pythonforbeginners.com/systems-programming/using-the-csv-module-in-python/
    ifile  = open(path, "r")
    reader = csv.reader(ifile)

    image_steering_map = {}

    rownum = 0
    for row in reader:
        # Save header row.
        if rownum == 0:
            header = row
        else:
            colnum = 0
            image_name = ''
            steering = 0.0
            for col in row:
                if colnum == 0:
                    image_name = get_image_name(col, 'IMG/(.*)')
#                     image_name = get_image_name(col, '(.*)')
                elif colnum == 3:
                    steering = float(col)
                colnum += 1
            image_steering_map[image_name] = steering

        rownum = rownum + 1
        
    return image_steering_map
        
image_steering_map = read_data_file('data/driving_log.csv')
# print(image_steering_map)
def bgr_to_rgb(img):
    b,g,r = cv2.split(img)
    img2 = cv2.merge([r,g,b])
    return img2

In [3]:
def read_image(path):
    images = []
    steering = []
    for img in glob.glob(path):
#         file_name = get_image_name(img, 'data/IMG/(.*)')
#         print(img)
        file_name = get_image_name(img, 'sample/(.*)')
#         print(image_steering_map[file_name])
        steering.append(image_steering_map[file_name])
        read_image = cv2.imread(img)
#         read_image = read_image.astype(np.float32)
        images.append(bgr_to_rgb(read_image))
        
    return images, steering

In [4]:
def convert_to_numpy_array(arr):
    return np.asarray(arr).astype(np.float32)

In [5]:
# images, steering = read_image('data/IMG/center*')
images, steering = read_image('sample/center*')
print(len(images))
print(len(steering))



3
3


In [ ]:
print(steering)

In [6]:
np_images = convert_to_numpy_array(images)
np_streering = convert_to_numpy_array(steering)

In [ ]:
print(len(images))

In [7]:
# comma.ai
def get_model(time_len=1):
  ch, row, col = 3, 160, 320  # camera format

  model = Sequential()
  model.add(Lambda(lambda x: x/127.5 - 1.,
            input_shape=(row, col, ch),
            output_shape=(row, col, ch)))
  model.add(Convolution2D(16, 8, 8, subsample=(4, 4), border_mode="same"))
  model.add(ELU())
  model.add(Convolution2D(32, 5, 5, subsample=(2, 2), border_mode="same"))
  model.add(ELU())
  model.add(Convolution2D(64, 5, 5, subsample=(2, 2), border_mode="same"))
  model.add(Flatten())
  model.add(Dropout(.2))
  model.add(ELU())
  model.add(Dense(512))
  model.add(Dropout(.5))
  model.add(ELU())
  model.add(Dense(1))

  model.compile(optimizer="adam", loss="mse")

  return model

In [ ]:
X_train, X_validate, y_train, y_validate = train_test_split(np_images, np_streering, test_size=0.33)
print(len(X_train))
print(len(X_validate))

In [ ]:
def data_iterator(x, y, size):
    """ A simple data iterator """
    while True:
        # shuffle labels and features
        idxs = np.arange(0, len(x))
        np.random.shuffle(idxs)
        print(idxs)
#         shuf_features = x[idxs]
#         shuf_labels = y[idxs]
        batch_size = size
        for batch_idx in range(0, len(x), batch_size):
            images_batch = x[batch_idx:batch_idx+batch_size]
            labels_batch = y[batch_idx:batch_idx+batch_size]
            yield (images_batch, labels_batch)

In [ ]:
BATCH_SIZE = 128
EPOCH = 5

model = get_model()
model.fit_generator(
data_iterator(X_train, y_train, BATCH_SIZE),
samples_per_epoch=len(X_train),
nb_epoch=EPOCH,
validation_data=data_iterator(X_validate, y_validate, BATCH_SIZE),
nb_val_samples=500
)

In [8]:
BATCH_SIZE = 128
EPOCH = 5

model = get_model()
history = model.fit(np_images, np_streering,
                    batch_size=BATCH_SIZE, nb_epoch=EPOCH,
                    verbose=1, validation_data=(np_images, np_streering))
arr = model.predict(np_images)
print(np_streering)
print(arr)
# score = model.evaluate(np_images, np_streering, verbose=0)
print(score)
# print('Test score:', score[0])
# print('Test accuracy:', score[1])

Train on 3 samples, validate on 3 samples
Epoch 1/5
3/3 [==============================] - 0s - loss: 0.0646 - val_loss: 4.7409
Epoch 2/5
3/3 [==============================] - 0s - loss: 4.5599 - val_loss: 12.4010
Epoch 3/5
3/3 [==============================] - 0s - loss: 7.0247 - val_loss: 4.2417
Epoch 4/5
3/3 [==============================] - 0s - loss: 7.0139 - val_loss: 0.7626
Epoch 5/5
3/3 [==============================] - 0s - loss: 1.8109 - val_loss: 0.6929
[ 0.         -0.0787459   0.57846057]
[[ 0.56638491]
 [-0.48750398]
 [-0.68278641]]


NameError: name 'score' is not defined

In [ ]:
print("Saving model weights and configuration file.")

if not os.path.exists("./outputs/steering_model"):
  os.makedirs("./outputs/steering_model")

model.save_weights("./outputs/steering_model/model.h5", True)
with open('./outputs/steering_model/model.json', 'w') as outfile:
    json.dump(model.to_json(), outfile)